## Import libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

## Import data

In [4]:
movies = pd.read_csv('movies.csv',
            sep=',',
            names=['MovieID','Title','Genres'],
            engine='python',
            header=0
            )

In [26]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
ratings = pd.read_csv('ratings.csv',
            sep=',',
            names=['UserID','MovieID','Rating', 'Timestamp'],
            parse_dates=['Timestamp'],
            engine='python',
            header=0
            )
ratings = ratings.drop("Timestamp", axis=1)

In [9]:
ratings.head()

,UserID,MovieID,Rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [10]:
movie_ratings = pd.merge(movies, ratings, on='MovieID')

In [11]:
movie_ratings.head()

,MovieID,Title,Genres,UserID,Rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,3.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13,5.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,3.0


In [14]:
movie_ratings.shape

(100004, 5)

In [62]:
genre = movie_ratings['Genres']
genre = genre.str.get_dummies()

In [63]:
genre.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [69]:
movie_ratings_genres = pd.concat([movie_ratings.drop(['Genres'], axis=1), genre], axis=1,)

In [71]:
movie_ratings_genres = movie_ratings_genres.drop(['(no genres listed)'], axis=1)

In [77]:
movie_ratings_genres.sample(5)

,MovieID,Title,UserID,Rating,Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
57266,2953,Home Alone 2: Lost in New York (1992),150,1.5,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
36532,1414,Mother (1996),537,4.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4588,150,Apollo 13 (1995),194,4.0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
34096,1300,My Life as a Dog (Mitt liv som hund) (1985),232,5.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
32670,1266,Unforgiven (1992),433,4.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [94]:
genres = list(pd.unique(movies["Genres"].str.split("|", expand=True).stack()))
genres.remove('(no genres listed)')
genres

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Mystery',
 'Sci-Fi',
 'Documentary',
 'IMAX',
 'War',
 'Musical',
 'Western',
 'Film-Noir']

# Melt and pivot with one hot encoding

In [98]:
#melting dataframe to form one hot encoding dataframe
mdf = pd.melt(movie_ratings_genres[['UserID', 'Rating'] + genres], id_vars=['UserID', 'Rating'], var_name='Genre')
mdf

,UserID,Rating,Genre,value
0,7,3.0,Adventure,1
1,9,4.0,Adventure,1
2,13,5.0,Adventure,1
3,15,2.0,Adventure,1
4,19,3.0,Adventure,1
...,...,...,...,...
1900071,287,5.0,Film-Noir,0
1900072,73,4.5,Film-Noir,0
1900073,611,5.0,Film-Noir,0
1900074,611,3.0,Film-Noir,0


In [99]:
#keeping only 1 values for every user at every genre
mdf = mdf[mdf['value']==1][['UserID', 'Rating', 'Genre']]
mdf

,UserID,Rating,Genre
0,7,3.0,Adventure
1,9,4.0,Adventure
2,13,5.0,Adventure
3,15,2.0,Adventure
4,19,3.0,Adventure
...,...,...,...
1896118,628,2.5,Film-Noir
1896119,664,3.5,Film-Noir
1896336,199,3.5,Film-Noir
1896502,380,2.5,Film-Noir


In [100]:
#pivoting the dataframe and calculating avarage rating for every genre by every user
movie_genres_avg_rating = pd.pivot_table(mdf, columns = ['Genre'], index = ['UserID'], values = ['Rating'], aggfunc = np.mean)
movie_genres_avg_rating = movie_genres_avg_rating.round(decimals=2)
movie_genres_avg_rating

Rating                                                              \
Genre  Action Adventure Animation Children Comedy Crime Documentary Drama   
UserID                                                                      
1        2.80      2.17      2.00     2.50   2.00  2.50         NaN  2.57   
2        3.32      3.75      3.67     3.50   3.09  3.67         NaN  3.62   
3        3.47      3.30      2.88     2.70   3.61  3.70        3.67  3.92   
4        4.27      4.34      4.71     4.63   4.33  4.16        4.00  4.46   
5        4.00      3.82      3.90     3.89   3.98  3.45        3.67  3.81   
...       ...       ...       ...      ...    ...   ...         ...   ...   
667      3.27      3.57      4.00     4.00   3.88  3.67         NaN  3.71   
668      3.50      3.00      3.00     3.00   3.44  4.57         NaN  4.31   
669      3.78      3.80       NaN      NaN   3.40  3.75         NaN  3.42   
670      3.60      3.40      4.00     4.00   3.86  4.00         NaN  3.76   
671      4.00      4.09      4.25     4.33   3.81  4.05        4.00  3.78   

                                                                               \
Genre  Fantasy Film-Noir Horror  IMAX Musical Mystery Romance Sci-Fi Thriller   
UserID                                                                          
1         2.12       NaN   3.00   NaN    3.00     NaN    3.50   2.40     2.83   
2         3.40       NaN   3.50  4.00    3.75    4.00    3.59   3.80     3.33   
3         3.00       NaN   3.38  2.88    2.50    3.75    3.65   3.14     3.55   
4         4.57       5.0   3.94  5.00    4.83    4.09    4.50   4.26     4.02   
5         3.75       NaN   3.88  4.00    4.25    3.33    4.08   4.00     3.62   
...        ...       ...    ...   ...     ...     ...     ...    ...      ...   
667       4.00       NaN    NaN  4.00    3.67    4.00    3.26   4.25     3.50   
668       4.00       NaN   5.00   NaN     NaN    3.00    5.00    NaN     4.50   
669       3.00       5.0   2.57   NaN     NaN    3.67    3.25   2.88     3.08   
670       3.33       1.5   5.00  3.00     NaN    3.75    3.71   3.75     3.90   
671       4.41       NaN   4.00  4.00    4.25    4.25    3.87   4.04     4.05   

                      
Genre    War Western  
UserID                
1       2.00     3.0  
2       3.80     5.0  
3       4.17     4.0  
4       4.75     3.0  
5       4.00     NaN  
...      ...     ...  
667     3.83     3.0  
668     4.50     NaN  
669      NaN     5.0  
670     4.33     1.0  
671     3.86     3.5  

[671 rows x 19 columns]

In [105]:
movie_genres_avg_rating = movie_genres_avg_rating.fillna(movie_genres_avg_rating.mean()).round(decimals=2)
movie_genres_avg_rating

Rating                                                              \
Genre  Action Adventure Animation Children Comedy Crime Documentary Drama   
UserID                                                                      
1        2.80      2.17      2.00     2.50   2.00  2.50        3.84  2.57   
2        3.32      3.75      3.67     3.50   3.09  3.67        3.84  3.62   
3        3.47      3.30      2.88     2.70   3.61  3.70        3.67  3.92   
4        4.27      4.34      4.71     4.63   4.33  4.16        4.00  4.46   
5        4.00      3.82      3.90     3.89   3.98  3.45        3.67  3.81   
...       ...       ...       ...      ...    ...   ...         ...   ...   
667      3.27      3.57      4.00     4.00   3.88  3.67        3.84  3.71   
668      3.50      3.00      3.00     3.00   3.44  4.57        3.84  4.31   
669      3.78      3.80      3.64     3.51   3.40  3.75        3.84  3.42   
670      3.60      3.40      4.00     4.00   3.86  4.00        3.84  3.76   
671      4.00      4.09      4.25     4.33   3.81  4.05        4.00  3.78   

                                                                               \
Genre  Fantasy Film-Noir Horror  IMAX Musical Mystery Romance Sci-Fi Thriller   
UserID                                                                          
1         2.12      3.91   3.00  3.76    3.00    3.75    3.50   2.40     2.83   
2         3.40      3.91   3.50  4.00    3.75    4.00    3.59   3.80     3.33   
3         3.00      3.91   3.38  2.88    2.50    3.75    3.65   3.14     3.55   
4         4.57      5.00   3.94  5.00    4.83    4.09    4.50   4.26     4.02   
5         3.75      3.91   3.88  4.00    4.25    3.33    4.08   4.00     3.62   
...        ...       ...    ...   ...     ...     ...     ...    ...      ...   
667       4.00      3.91   3.44  4.00    3.67    4.00    3.26   4.25     3.50   
668       4.00      3.91   5.00  3.76    3.59    3.00    5.00   3.52     4.50   
669       3.00      5.00   2.57  3.76    3.59    3.67    3.25   2.88     3.08   
670       3.33      1.50   5.00  3.00    3.59    3.75    3.71   3.75     3.90   
671       4.41      3.91   4.00  4.00    4.25    4.25    3.87   4.04     4.05   

                      
Genre    War Western  
UserID                
1       2.00    3.00  
2       3.80    5.00  
3       4.17    4.00  
4       4.75    3.00  
5       4.00    3.61  
...      ...     ...  
667     3.83    3.00  
668     4.50    3.61  
669     3.89    5.00  
670     4.33    1.00  
671     3.86    3.50  

[671 rows x 19 columns]

# K-means clustering

In [162]:
kmeans = KMeans(n_clusters=20, init='k-means++', max_iter=300, n_init=10, random_state=0)
predictions = kmeans.fit_predict(movie_genres_avg_rating)
user_cluster = pd.DataFrame(index=movie_genres_avg_rating.index)
user_cluster['Cluster'] = predictions
user_cluster.to_csv('user_cluster.csv')
user_cluster

c:\users\30694\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


,Cluster
UserID,
1,8
2,15
3,3
4,7
5,17
...,...
667,12
668,13
669,15


In [139]:
movie_ratings_cluster = pd.merge(movie_ratings, user_cluster, on='UserID')
movie_ratings_cluster

,MovieID,Title,Genres,UserID,Rating,Cluster
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,3.0,18
1,10,GoldenEye (1995),Action|Adventure|Thriller,7,3.0,18
2,21,Get Shorty (1995),Comedy|Crime|Thriller,7,3.0,18
3,31,Dangerous Minds (1995),Drama,7,3.0,18
4,34,Babe (1995),Children|Drama,7,4.0,18
...,...,...,...,...,...,...
99999,2385,Home Fries (1998),Comedy|Romance,227,2.0,13
100000,2386,Jerry Springer: Ringmaster (1998),Comedy|Drama,227,5.0,13
100001,2387,Very Bad Things (1998),Comedy|Crime,227,5.0,13
100002,2389,Psycho (1998),Crime|Horror|Thriller,227,5.0,13


In [140]:
movie_ratings_cluster = movie_ratings_cluster.drop(columns=['Genres', 'Title', 'UserID'], axis=1)
movie_ratings_cluster.
movie_ratings_cluster

,MovieID,Rating,Cluster
0,1,3.0,18
1,10,3.0,18
2,21,3.0,18
3,31,3.0,18
4,34,4.0,18
...,...,...,...
99999,2385,2.0,13
100000,2386,5.0,13
100001,2387,5.0,13
100002,2389,5.0,13


In [163]:
#pivot to calculate the average rating for every movie by every cluster
movies_cluster_avg = pd.pivot_table(udf, columns='Cluster', index='MovieID', values=['Rating'], aggfunc=np.mean)
movies_cluster_avg = movies_cluster_avg.round(decimals=2)
movies_cluster_avg.to_csv('movies_cluster_avg.csv')
movies_cluster_avg

Rating                                                              \
Cluster     0     1     2     3     4   5     6     7     8   9   10    11   
MovieID                                                                      
1         3.29  4.07  3.88  3.58  1.83 NaN  3.64  4.79  2.38 NaN NaN  2.50   
2         2.83  3.50  3.38  3.11  4.00 NaN  2.00  5.00  2.00 NaN NaN  2.75   
3         2.33  4.00   NaN  3.04  2.50 NaN  4.50  4.40  1.50 NaN NaN  3.00   
4         1.50   NaN  1.00  2.00  2.00 NaN   NaN  3.00   NaN NaN NaN   NaN   
5         2.70  3.40  3.00  2.80  3.17 NaN   NaN  4.33  2.75 NaN NaN  3.00   
...        ...   ...   ...   ...   ...  ..   ...   ...   ...  ..  ..   ...   
161944     NaN   NaN   NaN   NaN   NaN NaN   NaN  5.00   NaN NaN NaN   NaN   
162376     NaN   NaN   NaN  4.50   NaN NaN   NaN   NaN   NaN NaN NaN   NaN   
162542     NaN   NaN   NaN   NaN   NaN NaN  5.00   NaN   NaN NaN NaN   NaN   
162672     NaN   NaN   NaN   NaN   NaN NaN  3.00   NaN   NaN NaN NaN   NaN   
163949     NaN   NaN   NaN  5.00   NaN NaN   NaN   NaN   NaN NaN NaN   NaN   

                                                       
Cluster    12    13  14    15    16    17    18    19  
MovieID                                                
1        3.95  2.08 NaN  4.03  4.25  4.56  3.86  4.45  
2        3.46  3.38 NaN  3.58  3.93  3.80  3.00  3.67  
3        3.00  2.00 NaN  3.75  3.00  3.38  2.00  3.50  
4        3.25  1.00 NaN  3.00   NaN  3.00  3.00  3.00  
5        3.40   NaN NaN  3.50  4.12  3.50  3.12  3.25  
...       ...   ...  ..   ...   ...   ...   ...   ...  
161944    NaN   NaN NaN   NaN   NaN   NaN   NaN   NaN  
162376    NaN   NaN NaN   NaN   NaN   NaN   NaN   NaN  
162542    NaN   NaN NaN   NaN   NaN   NaN   NaN   NaN  
162672    NaN   NaN NaN   NaN   NaN   NaN   NaN   NaN  
163949    NaN   NaN NaN   NaN   NaN   NaN   NaN   NaN  

[9066 rows x 20 columns]